In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind_from_stats

In [2]:
#load data from excel into dataframe
df=pd.read_excel('OaO data modified.xlsx',header=[0,1])

In [ ]:
df.head()

In [ ]:
df.describe().T

plan going forward:
1. Get all means for each make (already done)
2. choose categorical vars to slice by
3. get all means for each make when sliced by categorical vars
4. perform ttest and sample size filter to determine validity
5. put results in table, perform hierarchical clustering

[var1, var2, mean, stDev, count1, count2]

In [3]:
catList = ['Age Bucket','Education - Principal Driver','Household Income','Intention Vehicle - Make','Principal Driver - Marital Status','Principal Driver - Sex','Purch Veh From Company Origin: Germany','Purch Veh From Company Origin: Japan','Purch Veh From Company Origin: Korea','Purch Veh From Company Origin: U.S.','Race Code (1)','Replaced Vehicle','State','Familiar With Categorical']
makeList=['Acura','Audi','BMW','Buick','Cadillac','Chevrolet','Chrysler','Dodge','Ford','GMC','Honda','Hyundai','Infiniti','Jeep','Kia','Lexus','Lincoln','Mazda','Mercedes-Benz','MINI','Mitsubishi','Nissan','RAM','Subaru','Tesla','Toyota','Volkswagen','Volvo']
optionExcludeList=['Kansas','Louisiana','Oklahoma','Arkansas','New Hampshire','Maine','New Mexico','Idaho','Mississippi','Rhode Island','West Virginia','Delaware','Nebraska','District Of Columbia','Montana','South Dakota','Vermont','North Dakota','Lincoln','Volvo','Infiniti','Mitsubishi','Mini','Land Rover','Fiat','Jaguar','Scion','Porsche','Mercury','Suzuki','American','RAM','Chevy','Bentley','Eagle','Daewoo','smart','Isuzu','Maserati','Alfa','Daihatsu','Ferrari','Aston','Peugeot','Oldsmobile','Plymouth','Mercedes-Benz','Saturn']

In [4]:
#initialize dfFull dict with full dataset
dfFull = {'Full': df.describe().T.drop(['min','25%','50%','75%','max'], axis=1)}

#add each subgroup dataframe to dfFull dict under subcategory key
for subset in catList:
    print(subset)
    if subset=='Familiar With Categorical':
        for make in makeList:
            for option in df[make,subset].dropna().unique():
                test=df[df[make,subset]==option].describe().T.drop(['min','25%','50%','75%','max'], axis=1)
                dfFull[make+'_'+subset+'_'+option]=test
    else:
        for option in df['Demographic',subset].dropna().unique():
            if option not in optionExcludeList:
                test=df[df['Demographic',subset]==option].describe().T.drop(['min','25%','50%','75%','max'], axis=1)
                dfFull[subset+'_'+option]=test


Age Bucket
Education - Principal Driver
Household Income
Intention Vehicle - Make
Principal Driver - Marital Status
Principal Driver - Sex
Purch Veh From Company Origin: Germany
Purch Veh From Company Origin: Japan
Purch Veh From Company Origin: Korea
Purch Veh From Company Origin: U.S.
Race Code (1)
Replaced Vehicle
State
Familiar With Categorical


In [ ]:
for k,v in dfFull.items():
    print(k)

In [ ]:
#iterate through each possible dataframe, concatenate new ttest frame to dfTest
import itertools
keyList=list(dfFull.keys())
dfTest=pd.DataFrame(index=dfFull['Full'].index, columns=['subset1', 'subset2', 'avg1', 'avg2', 'significance', 'likelyhood','count1', 'count2'])
for subset1,subset2 in itertools.combinations(keyList,2):
    temp=pd.DataFrame(index=dfFull['Full'].index, columns=['subset1', 'subset2', 'avg1', 'avg2', 'significance', 'likelyhood','count1', 'count2'])
    temp['subset1']=subset1
    temp['subset2']=subset2
    temp['avg1']=dfFull[subset1]['mean'].values
    temp['avg2']=dfFull[subset2]['mean'].values
    t,p=ttest_ind_from_stats(dfFull[subset1]['mean'],dfFull[subset1]['std'],dfFull[subset1]['count'],dfFull[subset2]['mean'],dfFull[subset2]['std'],dfFull[subset2]['count'],equal_var=False)
    temp['significance']=t
    temp['likelyhood']=p
    temp['count1']=dfFull[subset1]['count']
    temp['count2']=dfFull[subset2]['count']
    temp=temp[temp['count1']>30]
    temp=temp[temp['count2']>30]
    dfTest=pd.concat([dfTest,temp], axis=0)

D:\WinPython\python-3.5.3.amd64\lib\site-packages\scipy\stats\_distn_infrastructure.py:875: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
D:\WinPython\python-3.5.3.amd64\lib\site-packages\scipy\stats\_distn_infrastructure.py:875: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
D:\WinPython\python-3.5.3.amd64\lib\site-packages\scipy\stats\_distn_infrastructure.py:1814: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


In [ ]:
len(dfTest)

In [ ]:
dfSignificant=dfTest[dfTest['likelyhood']<.050001]

In [ ]:
dfSignificant

In [ ]:
dfSignificant.to_excel("significant.xlsx")